### Experiment settings

In [ ]:
from private_billing.server import TCPAddress

edge_address = TCPAddress("0.0.0.0", 5555)

In [ ]:
import zmq
from private_billing.messages import Message
from private_billing.server import PickleEncoder

ctxt = zmq.Context()
sock = ctxt.socket(zmq.REQ)


def send(msg: Message, target: TCPAddress):
    with sock.connect(str(target)):
        enc = PickleEncoder.encode(msg)
        sock.send(enc)
        repl = sock.recv()
    return PickleEncoder.decode(repl)

### Start bootstrap

In [ ]:
from src.experiment.experiment import BootStrapMessage

msg = BootStrapMessage(None)
resp = send(msg, edge_address)
print(resp)

### Start round

In [ ]:
from private_billing.messages import ContextMessage
from private_billing.core import CycleContext, vector

cyc_len = 744
cyc = CycleContext(
    0,
    cyc_len,
    vector.new(cyc_len, 0.21),
    vector.new(cyc_len, 0.05),
    vector.new(cyc_len, 0.11),
)
msg = ContextMessage(None, cyc)
resp = send(msg, edge_address)
print(resp)

### Get telemetry data

In [ ]:
from src.experiment.telemetry import GetTelemetryMessage, TelemetryMessage

msg = GetTelemetryMessage(None, 0)
resp: TelemetryMessage = send(msg, edge_address)

for val in resp.value:
    print(val)